## Chapter 02. Python速成
**只包含了本章的一些关键技术点**

### 2.1 基本内容 
#### collections.defaultdict
假设你需要计算某份文件中的单词数目。一个明显的方式是，建立一个键是单词、值是单词出现次数的字典。每次你查到一个单词，如果字典中存在这个词，就在该词的计数上增加1，如果字典中没有这个词，就把这个词增加到这个字典中：

In [1]:
document = ['alex', 'delia', 'tanya', 'alex', 'delia', 'delia', 'tanya', 'tanya', 'tanya']

# 方法1
word_counts = {} 
for word in document:
    if word in word_counts:
        word_counts[word] += 1
    else:
        word_counts[word] = 1
        
word_counts

{'alex': 2, 'delia': 3, 'tanya': 4}

In [2]:
# 方法2
word_counts = {} 
for word in document: 
    try:
        word_counts[word] += 1 
    except KeyError:
        word_counts[word] = 1
    
word_counts

{'alex': 2, 'delia': 3, 'tanya': 4}

In [3]:
# 方法3
word_counts = {} 
for word in document:
    previous_count = word_counts.get(word, 0)
    word_counts[word] = previous_count + 1

word_counts

{'alex': 2, 'delia': 3, 'tanya': 4}

以上三种方法都略显笨拙，这是defaultdict的意义之所在。 一个defaultdict相当于一个标准的字典，除了当你查找一个没有包含在内的键时，它用一个你提供的零参数函数建立一个新的键，并为它的值增加1：

In [4]:
from collections import defaultdict

word_counts = defaultdict(int) # int()生成0
for word in document:
    word_counts[word] += 1
    
word_counts

defaultdict(int, {'alex': 2, 'delia': 3, 'tanya': 4})

#### collections.Counter
一个计数器将一个序列的值转化成一个类似于整型的标准字典（即defaultdict(int)）的键到计数的对象映射。我们主要用它来生成直方图：

In [5]:
from collections import Counter

word_counts = Counter(document)
word_counts

Counter({'alex': 2, 'delia': 3, 'tanya': 4})

In [6]:
word_counts.most_common()

[('tanya', 4), ('delia', 3), ('alex', 2)]

### 2.2 进阶内容
#### 排序
每个Python列表都有一个sort()方法可以恰当地排序。如果你不想弄乱你的列表，可以使用sorted()函数，它会返回一个新列表

#### 列表解析
我们有时可能会想把一个列表转换为另一个列表，例如只保留其中一些元素，或更改其中 一些元素，或者同时做这两种变动。可以执行这种操作的 Python 技巧叫作列表解析(list comprehension):

In [7]:
even_numbers = [x for x in range(5) if x%2 ==  0]
even_numbers

[0, 2, 4]

In [8]:
squares = [x * x for x in range(5)]
squares

[0, 1, 4, 9, 16]

In [9]:
square_dict = {x : x * x for x in range(5)}
square_dict

{0: 0, 1: 1, 2: 4, 3: 9, 4: 16}

In [10]:
square_set = {x * x for x in [1, -1]}
square_set

{1}

如果你不需要来自原列表中的值，常规的方式是使用下划线作为变量：

In [11]:
zeroes = [0 for _ in even_numbers]
zeroes

[0, 0, 0]

列表解析可以包括多个for语句：

In [12]:
pairs = [(x, y) 
         for x in range(3)
         for y in range(3)]

pairs

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]

其中后面的for语句可以使用前面的for语句的结果：

In [13]:
pairs = [(x, y) 
         for x in range(3)
         for y in range(3+x)]

pairs

[(0, 0),
 (0, 1),
 (0, 2),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4)]

#### 生成器和迭代器
列表的一个问题是它很容易变得非常大。range(1000000)能创建一个有100万个元素的列表：如果你需要每次只处理其中一个元素，这将会是极大的资源浪费（或会导致内存不足）；如果你只需要前面的几个值，那对整个列表都进行计算也是一种浪费。

生成器（generator）是一种可以对其进行迭代（对我们来说，通常使用for语句）的程序，但是它的值只按需延迟（lazily）产生。

In [14]:
def lazy_range(n):
    """a lazy version of range""" 
    i = 0 
    while i < n:
        yield i
        i += 1

for i in lazy_range(5): 
    print(i)

0
1
2
3
4


Python确实有一个和lazy_range()一样的函数， 叫作xrange()， 并且在Python 3中，range()函数本身就是延迟的。

#### 函数式工具
在传递函数的时候，有时我们可能想部分地应用（或curry）函数来创建新函数。 下面是一个简单的例子，假设我们有一个含两个变量的函数：

In [15]:
def exp(base, power):
    return base ** power

exp(2, 10)

1024

我们想用它来创建一个单变量的函数是exp(2, power)的结果：

In [16]:
from functools import partial 
two_to_the = partial(exp, 2) 
two_to_the(10)

1024

偶尔我们也会使用函数map()、reduce()和filter()，它们为列表解析提供了函数式替换方案：

In [17]:
def double(x): 
    return 2 * x

xs = [1, 2, 3, 4]

twice_xs = [double(x) for x in xs]
twice_xs

[2, 4, 6, 8]

In [18]:
ret = map(double, xs)
list(ret)

[2, 4, 6, 8]

In [19]:
def is_even(x):
    """True if x is even, False if x is odd""" 
    return x % 2 == 0

ret = [x for x in xs if is_even(x)] 
ret = filter(is_even, xs)
list(ret)

[2, 4]

In [20]:
def multiply(x, y): 
    return x * y

from functools import reduce
ret = reduce(multiply, xs)
ret

24

In [21]:
# ((((((1+2)+3)+4)+5)+6))
reduce(lambda x, y: x + y, [2, 3, 4, 5, 6], 1)

21

#### 枚举
有时候，你可能想在一个列表上迭代，并且同时使用它的元素和元素的索引。熟悉其它语言的程序员可能使用各种风格的枚举。Python则有自己独特的风格：

```python
# 非Python用法 
for i in range(len(documents)):
    # document = documents[i]
    # do_something(i, document)

# 也非Python用法 
i = 0 
for document in documents:
    # do_something(i, document)
    i += 1
```

Python惯用的解决方案是使用枚举（enumerate），它会产生(index, element)元组：
```python
for i, document in enumerate(documents):
    do_something(i, document)

for i, _ in enumerate(documents):
    pass
```

#### 压缩和参数拆分
如果想把两个或多个列表压缩在一起，可以使用zip()把多个列表转换为一个对应元素的元组的单个列表中：

In [22]:
list1 = ['a', 'b', 'c'] 
list2 = [1, 2, 3] 
ret = zip(list1, list2)
list(ret)

[('a', 1), ('b', 2), ('c', 3)]

可以使用一种特殊的方法“解压”一个列表，星号执行参数拆分（argument unpacking）：

In [23]:
pairs = [('a', 1), ('b', 2), ('c', 3)] 
letters, numbers = zip(*pairs)
letters, numbers

(('a', 'b', 'c'), (1, 2, 3))

你可以在任何函数上使用参数拆分：

In [24]:
def add(a, b): return a + b

add(1, 2)    # 返回3
# add([1, 2])  # TypeError!
add(*[1, 2]) # 返回3

3

#### args 和 kwargs
假如我们想创建一个更高阶的函数，把某个函数f()作为输入，并返回一个对任意输入都返回f()值两倍的新函数：

In [25]:
def doubler(f):
    def g(x):
        return 2 * f(x) 
    return g

def f1(x):
    return x + 1

g = doubler(f1)

In [26]:
g(3)

8

In [27]:
g(-1)

0

但对于有多个参数的函数来说，就不适用：

In [28]:
def f2(x, y): 
    return x + y

g = doubler(f2)

# TypeError: g()只能有一个参数（给定了两个）
# g(1, 2) 

我们所需要的是一种指定一个可以取任意参数的函数的方法，利用参数拆分和一点点魔法就可以做到这一点：

In [29]:
def magic(*args, **kwargs): 
    print("unnamed args:", args)
    print("keyword args:", kwargs)

magic(1, 2, key="word", key2="word2")

unnamed args: (1, 2)
keyword args: {'key': 'word', 'key2': 'word2'}


也就是说，当我们定义了这样一个函数时，args是一个它的未命名参数的元组，而kwargs是一个它的已命名参数的dict 。 反过来也适用，你可以使用一个list（或者tuple）和dict来给函数提供参数：

In [30]:
def other_way_magic(x, y, z):
 return x + y + z

x_y_list = [1, 2] 
z_dict = { "z" : 3 }

other_way_magic(*x_y_list, **z_dict)

6

In [31]:
def f2(x, y): 
    return x + y

def doubler_correct(f):
    def g(*args, **kwargs):
        return 2 * f(*args, **kwargs) 
    return g 

g = doubler_correct(f2)
g(1, 2)

6